# Project Code
#### Joey Livorno | gil15@pitt.edu | 2.24.2020

The purpose of this project is to process the contents of Donald Trump's Twitter feed and make linguistic and statistical discoveries based on the data, specifically using sentiment analysis.

These are the required libraries for this assignment:

In [1]:
#import libraries
import numpy as np
import pandas as pd
import re
from textblob import TextBlob as tb ##TextBlob object will allow for quick sentiment analysis
from emo_unicode import EMO_UNICODE #A set of dictionaries of emoticons : their text represenstations
from emo_unicode import UNICODE_EMO ## courtesy of NeelShah18 on GitHub                              

## Basic Data Processing

This section of the code will focus on reading in the twitter data and manipulating it into a much more workable object than simply the raw .csv file.

The first step is to read the .csv file into a pandas dataframe:

In [2]:
#read tweet information into dataframe
tweets = pd.read_csv('../data/tweets.csv')
tweets.head() #preview the df

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,RT @SenBillCassidy: January #JobsReport:✅22500...,02-21-2020 16:06:40,896.0,0.0,True,1.230887e+18
1,Twitter for iPhone,RT @SteveDaines: Obama sure didn’t build this ...,02-21-2020 16:06:20,2086.0,0.0,True,1.230886e+18
2,Twitter for iPhone,RT @JohnBoozman: Our servicemembers stand read...,02-21-2020 16:06:10,521.0,0.0,True,1.230886e+18
3,Twitter for iPhone,RT @RoyBlunt: ▶️ Unemployment is at a nearly 5...,02-21-2020 16:05:52,623.0,0.0,True,1.230886e+18
4,Twitter for iPhone,RT @JimInhofe: Happy 79th birthday to @USCGRes...,02-21-2020 16:05:25,533.0,0.0,True,1.230886e+18


The next step is to make the text full parseable. Right now, it is not because the tweets contain emojis. We can use the emoji dictionaries and this method to convert all of the emojis to text:

(right now this does not function correctly, so for now we will keep the emojis in the text)

In [3]:
#replace emojis with their text representation
##this method is courtesy of kaggle.com user SRK

UNICODE_EMO = {v: k for k, v in EMO_UNICODE.items()}

def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = re.sub(r'('+emot+')', "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()), text)
    return text

Next we will use the TextBlob library to assign each of the tweets a polarity and subjectivity value. Polarity is on a scale from -1 to 1 and describes how positive or negative the tweet is, -1 being negative and 1 being positive. Subjectivity is on a scale from 0 to 1 and describes whether or not the text is presented as a fact or an opinion. For this measure, 0 would be objective information and 1 would be subjective information.

In [4]:
#create series objects of the polarity and subjectivity of the tweets
polarity = tweets.text.map(lambda x: tb(x).sentiment.polarity)
subjectivity = tweets.text.map(lambda x: tb(x).sentiment.subjectivity)

Then we will create new columns for each of the new series:

In [5]:
#add new columns to tweets df corresponding to the new info
tweets['polarity'] = polarity
tweets['subjectivity'] = subjectivity
tweets.head() #preview the df

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str,polarity,subjectivity
0,Twitter for iPhone,RT @SenBillCassidy: January #JobsReport:✅22500...,02-21-2020 16:06:40,896.0,0.0,True,1.230887e+18,0.068182,0.377273
1,Twitter for iPhone,RT @SteveDaines: Obama sure didn’t build this ...,02-21-2020 16:06:20,2086.0,0.0,True,1.230886e+18,0.625000,0.888889
2,Twitter for iPhone,RT @JohnBoozman: Our servicemembers stand read...,02-21-2020 16:06:10,521.0,0.0,True,1.230886e+18,0.200000,0.500000
3,Twitter for iPhone,RT @RoyBlunt: ▶️ Unemployment is at a nearly 5...,02-21-2020 16:05:52,623.0,0.0,True,1.230886e+18,0.100000,0.400000
4,Twitter for iPhone,RT @JimInhofe: Happy 79th birthday to @USCGRes...,02-21-2020 16:05:25,533.0,0.0,True,1.230886e+18,0.750000,0.750000


Since we are going to be looking at change over time, we will need a way to group the tweets chronologically. Luckily, the timestamp is included in the original csv. The most logical grouping seems to be by year though, so let's make a new column that isolates that value:

In [6]:
#make new column that shows year
tweets['created_at'] = pd.to_datetime(tweets['created_at']) #convert created_at column to timestamp data type
tweets['year'] = tweets['created_at'].dt.year.astype('Int64') #store the dates in a new column, isolating the year
tweets.head() #preview df

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str,polarity,subjectivity,year
0,Twitter for iPhone,RT @SenBillCassidy: January #JobsReport:✅22500...,2020-02-21 16:06:40,896.0,0.0,True,1.230887e+18,0.068182,0.377273,2020
1,Twitter for iPhone,RT @SteveDaines: Obama sure didn’t build this ...,2020-02-21 16:06:20,2086.0,0.0,True,1.230886e+18,0.625000,0.888889,2020
2,Twitter for iPhone,RT @JohnBoozman: Our servicemembers stand read...,2020-02-21 16:06:10,521.0,0.0,True,1.230886e+18,0.200000,0.500000,2020
3,Twitter for iPhone,RT @RoyBlunt: ▶️ Unemployment is at a nearly 5...,2020-02-21 16:05:52,623.0,0.0,True,1.230886e+18,0.100000,0.400000,2020
4,Twitter for iPhone,RT @JimInhofe: Happy 79th birthday to @USCGRes...,2020-02-21 16:05:25,533.0,0.0,True,1.230886e+18,0.750000,0.750000,2020


Now, the data is much more workable; we have the measurements we are looking to compare neatly organized within their corresponding rows, and we have a way of grouping the data chronologically. The last step is to create subgroups for different political issues that we can compare to:

(these are subject to change)

In [7]:
#create the subsets for each topic
russia = tweets['text'].str.contains('russia|Russia|moscow|Moscow|putin|Putin')
iran = tweets['text'].str.contains('iran|Iran|tehran|Tehran|Nuclear Deal|Rouhani')
nkorea = tweets['text'].str.contains('north korea|North Korea|DPRK|Pyongyang|pyongyang|Jong-Un|Jong-Il')

And we can take a peek as to how much data we've collected in each subset:

In [11]:
tweets[russia].describe()

,retweet_count,favorite_count,id_str,polarity,subjectivity,year
count,208.000000,208.000000,2.080000e+02,208.000000,208.000000,208.000000
mean,6643.052885,18472.812500,6.683236e+17,0.115942,0.461429,2015.341346
std,9739.939267,34803.918654,3.151860e+17,0.263328,0.251401,2.392384
min,6.000000,0.000000,1.393782e+17,-1.000000,0.000000,2011.000000
25%,33.750000,18.000000,3.918958e+17,0.000000,0.360000,2013.000000
50%,603.500000,71.000000,5.758315e+17,0.130353,0.507007,2015.000000
75%,11040.250000,18895.000000,8.840125e+17,0.270568,0.574316,2017.000000
max,63789.000000,163688.000000,1.230879e+18,0.800000,1.000000,2020.000000


In [10]:
tweets[iran].describe()

,retweet_count,favorite_count,id_str,polarity,subjectivity,year
count,242.000000,242.000000,2.420000e+02,242.000000,242.000000,242.000000
mean,9505.904959,31601.942149,7.180197e+17,0.058855,0.443805,2015.830579
std,19560.258206,89813.338876,3.674870e+17,0.276805,0.265545,2.968754
min,2.000000,0.000000,8.935366e+16,-1.000000,0.000000,2011.000000
25%,52.000000,13.250000,4.263230e+17,-0.018750,0.300000,2014.000000
50%,1060.500000,87.500000,6.257791e+17,0.065341,0.491083,2015.000000
75%,10327.750000,4839.000000,1.213077e+18,0.202159,0.573229,2020.000000
max,158004.000000,764333.000000,1.230515e+18,1.000000,1.000000,2020.000000


In [12]:
tweets[nkorea].describe()

,retweet_count,favorite_count,id_str,polarity,subjectivity,year
count,49.000000,49.000000,4.900000e+01,49.000000,49.000000,49.000000
mean,16490.367347,63514.224490,7.732103e+17,0.166546,0.503083,2016.183673
std,13581.745497,50437.836564,2.603149e+17,0.261091,0.199599,1.810542
min,7.000000,0.000000,3.082059e+17,-0.500000,0.000000,2013.000000
25%,6455.000000,29.000000,8.160579e+17,0.000000,0.416667,2017.000000
50%,14601.000000,66215.000000,8.820626e+17,0.182734,0.512876,2017.000000
75%,24525.000000,94969.000000,9.043771e+17,0.277000,0.580426,2017.000000
max,51684.000000,171720.000000,1.208045e+18,1.000000,1.000000,2019.000000
